In [3]:
# Add parent directory
from sys import path as sysPath
sysPath.append('../')

# Add custom modules
from Support_Code import general_module as gm
from Support_Code import info_module as im
from Image_Creator import main_image_creator as ic
from Score_Analysis import main_score_analysis as sa

sysPath.append('../Machine_Score/')
from Machine_Score import main_machine_score as ms
import main_SIMR as simr


# General modules
from os import path
import numpy as np
from matplotlib import pyplot as plt

if 'Machine_Compare' not in sysPath:
    sysPath.append('Machine_Compare')

from Image_Creator import main_image_creator as ic
import main_SIMR as simr

sa.test()
gm.test()
im.test()
ic.test()
ms.test()
simr.test()

SA: Hi!  You're in Matthew's Main program for score analysis!
GM: Hi!  You're in Matthew's module for generally useful functions and classes
IM: Hi!  You're in Matthew Ogden's information module for SPAM
IC: Hi!  You're in Matthew's main code for all things image creation.
MS: Hi!  You're in Matthew's SIMR module for all things machine scoring images
SIMR: Hi!  You're in Matthew's main program for all things galaxy collisions


In [4]:
#tDir = '/home/mbo2d/galStuff/spam_data_pl3/587722984435351614/' 
#tDir = '../../targetDir/'
tDir = '../../1237678620102623480/'

tInfo = im.target_info_class( targetDir = tDir, printAll=True)

if tInfo.status == False:
    print("WARNING")
else:
    print("Target Good!")

IM: target_info_class.__init__:
	 - targetDir:  ../../1237678620102623480/
IM: Target.initTargetDir():
	 - targetDir: ../../1237678620102623480/
NEW PATH EXISTS: /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/1237678620102623480/zoo_merger_models/
	 - targetDir: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/1237678620102623480/
	 - infoDir: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/1237678620102623480/information/
	 - baseInfoLoc: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/1237678620102623480/information/base_target_info.json
	 - allInfoLoc: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/1237678620102623480/information/target_info.json
	 - zooMergerDir: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/1237678620102623480/zoo_merger_models/
	 - plotDir: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/1237678620102623480/plots/
IM: Target: Opening target info json
Target Good!


In [5]:

print(tInfo.get('zoo_merger_models').get('00001'))

rId = 'r00001'
runArg = gm.inArgClass()
runArg.setArg("printAll",True)

rInfo = tInfo.getRunInfo( rID=rId, rArg = runArg)
rInfo.tInfo = tInfo

if rInfo.status == False:
    print("WARNING")
else:
    print("Run '%s' Good!"%rInfo.get('run_id'))

None
IM: run_info_class.__init__
	 - runDir:  /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/1237678620102623480/zoo_merger_models/run_00001/
IM: run.initRunDir
	 - runDir: /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/1237678620102623480/zoo_merger_models/run_00001/
	 - runDir: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/1237678620102623480/zoo_merger_models/run_00001/
	 - ptsDir: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/1237678620102623480/zoo_merger_models/run_00001/particle_files/
	 - imgDir: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/1237678620102623480/zoo_merger_models/run_00001/model_images/
	 - miscDir: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/1237678620102623480/zoo_merger_models/run_00001/misc_images/
	 - infoLoc: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/1237678620102623480/zoo_merger_models/run_00001/info.json
	 - baseLoc: (True) /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/1237678620102623480/zoo_merger_models/run_00001/base_i

In [10]:
pLoc = gm.validPath('../param/start_param.json')
print(pLoc)
newParams = im.group_score_parameter_class(pLoc = pLoc)
gm.pprint(newParams.group)
print("Done")

/mnt/c/Users/ogden/OneDrive/Desktop/galStuff/galaxyJSPAM/param/start_param.json
{'base_perturbation': {'cmpArg': {'cmpMethod': 'correlation'},
                       'imgArg': {'name': 'default',
                                  'pType': 'default'},
                       'name': 'base_perturbation',
                       'scoreType': 'perturbation',
                       'simArg': {'nPts': '100k',
                                  'name': '100k'}},
 'base_score': {'cmpArg': {'cmpMethod': 'correlation'},
                'imgArg': {'name': 'default',
                           'pType': 'default'},
                'name': 'base_score',
                'scoreType': 'target',
                'simArg': {'nPts': '100k',
                           'name': '100k'},
                'targetName': 'zoo'}}
Done


In [9]:
runArgs = gm.inArgClass()
runArgs.printArg()
runArgs.setArg('scoreParams', newParams.group)
runArgs.setArg('overWrite', True)
#runArgs.setArg('printAll', True)
runArgs.setArg('newScore',True)
runArgs.setArg('newInfo',True)
runArgs.setArg('runDir',rInfo.get('runDir'))
runArgs.runDir = rInfo.runDir
runArgs.setArg('tInfo',tInfo)

simr.simr_run( arg = runArgs, )
print('\n', tInfo.get('zoo_merger_models')[rInfo.get('run_id')])


GM: Printing Input arguments
	- printBase - <class 'bool'> :  True
	- printAll - <class 'bool'> :  False
	- nProc - <class 'int'> :  -1
	- simple - <class 'bool'> :  False
	- runDir - <class 'NoneType'> :  None
	- sdssDir - <class 'NoneType'> :  None
	- targetDir - <class 'NoneType'> :  None
	- dataDir - <class 'NoneType'> :  None
SIMR.pipelineRun: Inputs
	 - rDir: /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/targetDir/gen000/run_00001/
	 - rInfo: <class 'NoneType'>
IM: run_info_class.__init__
	 - runDir:  /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/targetDir/gen000/run_00001/
SIMR.pipelineRun: 
	 - rInfo:  <Support_Code.info_module.run_info_class object at 0x7fc108f40160>
SIMR: run: scores before
IM: run_info_class.printScores()
	 - run_id: 00001
	 - zoo_merger: 0.941176
	 - machine_scores: 0
WORKING: 
 {'run_id': '00001', 'model_data': '-9.93853,-4.5805,20.84411,-1.74126,-2.7698,6.11683,27.02299,23.08542,3.49716,4.74238,299.83308,24.48166,45.94874,302.74563,0.3,0.3,0.0,0.0,0.0,0.0,0

In [10]:
print('\n', tInfo.get('zoo_merger_models')[rInfo.get('run_id')])



 {'run_id': '00001', 'model_data': '-9.93853,-4.5805,20.84411,-1.74126,-2.7698,6.11683,27.02299,23.08542,3.49716,4.74238,299.83308,24.48166,45.94874,302.74563,0.3,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.94117,0,0,0.21251,-3.49999,5.5938,0.0,0.0,1.0,1.0,0.0,0.0', 'zoo_merger_score': 0.9411764705882353, 'machine_scores': {'base0_corr': 0.7376331544200321, 'base0_diff': 0.9468803106569776, 'base0_diff_scale': 0.4688031065697763, 'base0_diff_scale1': 0.8937606213139553, 'base0_diff_scale2': 0.8229343688565922, 'base0_overlap0': 0.5054912919157786, 'base0_overlap1': 0.3655356644471911, 'base0_overlap2': 0.4603863940468086, 'base0_overlap3': 0.46573162608245133, 'base0_bcorr0': 0.6641861873885665, 'base0_bcorr1': 0.5711423657294602, 'base0_bcorr2': 0.6404602265891284, 'base0_bcorr3': 0.6404418317056928}}


In [5]:
runArgs = gm.inArgClass()
runArgs.printArg()
runArgs.setArg('scoreParams', newParams.group)
runArgs.setArg('overWrite', True)
runArgs.setArg('printAll', False)
runArgs.setArg('printBase',False)
runArgs.setArg('newScore',True)
runArgs.runDir = rInfo.runDir
runArgs.setArg('tInfo',tInfo)

GM: Printing Input arguments
	- printBase - <class 'bool'> :  True
	- printAll - <class 'bool'> :  False
	- nProc - <class 'int'> :  -1
	- simple - <class 'bool'> :  False
	- runDir - <class 'NoneType'> :  None
	- sdssDir - <class 'NoneType'> :  None
	- targetDir - <class 'NoneType'> :  None
	- dataDir - <class 'NoneType'> :  None


In [6]:
import timeit

runArgs = gm.inArgClass()
runArgs.printArg()
runArgs.setArg('scoreParams', newParams.group)
runArgs.setArg('overWrite', False)
runArgs.setArg('printAll', False)
runArgs.setArg('printBase',False)
runArgs.setArg('newScore',True)
runArgs.runDir = rInfo.runDir
runArgs.setArg('tInfo',tInfo)

start = timeit.default_timer()

#Your statements here
n = 100
for i in range(n):
    simr.simr_run( arg = runArgs, rInfo=None  )
    print('%d / %d' % (i,n), end='\r')

stop = timeit.default_timer()

run_time = (stop - start) / n

print('Time_per_run: ', run_time)  

GM: Printing Input arguments
	- printBase - <class 'bool'> :  True
	- printAll - <class 'bool'> :  False
	- nProc - <class 'int'> :  -1
	- simple - <class 'bool'> :  False
	- runDir - <class 'NoneType'> :  None
	- sdssDir - <class 'NoneType'> :  None
	- targetDir - <class 'NoneType'> :  None
	- dataDir - <class 'NoneType'> :  None
WORKING: 
 {'run_id': '00001', 'model_data': '-9.93853,-4.5805,20.84411,-1.74126,-2.7698,6.11683,27.02299,23.08542,3.49716,4.74238,299.83308,24.48166,45.94874,302.74563,0.3,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.94117,0,0,0.21251,-3.49999,5.5938,0.0,0.0,1.0,1.0,0.0,0.0', 'zoo_merger_score': 0.9411764705882353, 'machine_scores': {'base0_corr': 0.7376331544200321, 'base0_diff': 0.9468803106569776, 'base0_diff_scale': 0.4688031065697763, 'base0_diff_scale1': 0.8937606213139553, 'base0_diff_scale2': 0.8229343688565922, 'base0_overlap0': 0.5054912919157786, 'base0_overlap1': 0.3655356644471911, 'base0_overlap2': 0.4603863940468086, 'base0_overlap3': 0.46573162608245133, 'b